# Few-shot classification via Prototypes

#### Preprocessing 

In [5]:
# Imports
import json
import numpy as np
from sklearn.metrics import accuracy_score
from scipy.special import softmax

In [6]:
# Reads in an annotation file from filename
# Returns list of class_ids from the file
def get_classes(filename):
    with open(filename, "r") as f:
        lines = f.readlines()
    return [int(line.split()[2]) for line in lines]

# Extracts the numpy features from a json file
# Output is a NxD array, where N is the number of examples and D is the 
# extracted feature size
def get_features(filename):
    with open(filename, "r") as f:
        data = json.load(f)
        arr = np.asarray(data)
    return arr

In [8]:
# Get classes from the annotation files
DATASET_PATH = '/vision/group/ntu-rgbd/'
supp_ann_filename = DATASET_PATH + 'few_shot_depth_support_ann.txt'
test_ann_filename = DATASET_PATH + '50_few_shot_depth_support_val_ann.txt'
supp_classes = get_classes(supp_ann_filename)
test_classes = get_classes(test_ann_filename) 

feature_dir = 'features/contrastive/'

## Get RGB Results

#### Get Prototype values for each class

In [9]:
# Load embeddings into numpy array
rgb_supp_embeds = get_features(feature_dir + 'rgb_support_embeds.json')
class2embeds = {}
for c, feat in zip(supp_classes, rgb_supp_embeds):
    if c not in class2embeds:
        class2embeds[c] = []
    class2embeds[c].append(feat)
class_ids = [key for key in class2embeds.keys()]
for c in class_ids:
    class2embeds[c] = np.asarray(class2embeds[c])
    class2embeds[c] = np.mean(class2embeds[c], axis=0)

#### Classify examples

In [10]:
test_embeds = get_features(feature_dir + 'rgb_test_embeds.json')
# Classify based on the closest score
preds = []
rgb_scores = []
for embed in test_embeds:
    scores = []
    for c in class_ids:
        score = embed @ class2embeds[c].T
        scores.append(score)
    rgb_scores.append(softmax(scores)) 
    class_pred = class_ids[np.argmax(scores)]
    preds.append(class_pred)

#### Get results

In [11]:
print("Accuracy:", accuracy_score(preds, test_classes))

Accuracy: 0.204


## Get Depth Results

#### Get Prototype values for each class

In [12]:
# Load embeddings into numpy array
depth_supp_embeds = get_features(feature_dir + 'depth_support_embeds.json')
class2embeds = {}
for c, feat in zip(supp_classes, depth_supp_embeds):
    if c not in class2embeds:
        class2embeds[c] = []
    class2embeds[c].append(feat)
class_ids = [key for key in class2embeds.keys()]
for c in class_ids:
    class2embeds[c] = np.asarray(class2embeds[c])
    class2embeds[c] = np.mean(class2embeds[c], axis=0)

#### Classify examples

In [13]:
test_embeds = get_features(feature_dir + 'depth_test_embeds.json')
# Classify based on the closest score
preds = []
depth_scores = []
for embed in test_embeds:
    scores = []
    for c in class_ids:
        score = embed @ class2embeds[c].T
        scores.append(score)
    depth_scores.append(softmax(scores)) 
    class_pred = class_ids[np.argmax(scores)]
    preds.append(class_pred)

#### Get results

In [14]:
print("Accuracy:", accuracy_score(preds, test_classes))

Accuracy: 0.196


## Get Fusion Results

In [15]:
# Combine the softmax outputs from the previous two examples + report new values
rgb_weight, depth_weight = 1.0, 1.0
rgb_scores = np.asarray(rgb_scores)
depth_scores = np.asarray(depth_scores)
fusion_scores = rgb_weight * rgb_scores + depth_weight * depth_scores / 2 # Simple averaging

In [16]:
preds = []
for score in fusion_scores:
    class_pred = class_ids[np.argmax(score)]
    preds.append(class_pred)

In [17]:
print("Accuracy:", accuracy_score(preds, test_classes))

Accuracy: 0.204
